In [1]:
import pandas as pd
file_path = "0515.csv"
df = pd.read_csv(file_path)
df.head()

,元智大學 1132(CM254C),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,下載日期:2025/5/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,點名日期,課號,課名,節次,學號,姓名,經度,緯度,簽到時間
3,20250307,1132-CM254C,機器學習應用,第 02 節,1101444,徐英豈,24.968098,121.26738,45723.37974
4,20250307,1132-CM254C,機器學習應用,第 02 節,1102455,江毓婷,24.970192,121.263435,45723.39526


In [2]:
# 重新設置標題行，並刪除不必要的行
df_cleaned = df.iloc[2:].reset_index(drop=True)
df_cleaned.columns = df_cleaned.iloc[0]  # 設置新標題
df_cleaned = df_cleaned[1:].reset_index(drop=True)  # 刪除舊標題行

# 重命名關鍵欄位
df_cleaned = df_cleaned.rename(columns={"姓名": "Name", "經度": "Latitude", "緯度": "Longitude", "簽到時間": "Check-in Time"})

# 轉換數據類型
df_cleaned["Latitude"] = pd.to_numeric(df_cleaned["Latitude"], errors="coerce")
df_cleaned["Longitude"] = pd.to_numeric(df_cleaned["Longitude"], errors="coerce")

# 轉換簽到時間（Excel 時間格式轉換）
df_cleaned["Check-in Time"] = pd.to_numeric(df_cleaned["Check-in Time"], errors="coerce")
df_cleaned["Check-in Time"] = pd.to_datetime(df_cleaned["Check-in Time"], unit="D", origin="1899-12-30")

df_cleaned

,點名日期,課號,課名,節次,學號,Name,Latitude,Longitude,Check-in Time
0,20250307,1132-CM254C,機器學習應用,第 02 節,1101444,徐英豈,24.968098,121.267380,2025-03-07 09:06:49.535999744
1,20250307,1132-CM254C,機器學習應用,第 02 節,1102455,江毓婷,24.970192,121.263435,2025-03-07 09:29:10.463999744
2,20250307,1132-CM254C,機器學習應用,第 02 節,1102514,王 杉,24.976917,121.255814,2025-03-07 09:18:27.648000256
3,20250307,1132-CM254C,機器學習應用,第 02 節,1102527,簡佳媛,25.021704,121.255318,2025-03-07 09:12:55.872000000
4,20250307,1132-CM254C,機器學習應用,第 02 節,1103002,林郁綺,24.970827,121.267715,2025-03-07 09:18:57.024000256
...,...,...,...,...,...,...,...,...,...
209,20250509,1132-CM254C,機器學習應用,第 02 節,1129918,李庭儀,24.970461,121.267509,2025-05-09 09:26:48.768000256
210,20250509,1132-CM254C,機器學習應用,第 03 節,1122810,江佳真,24.970739,121.268112,2025-05-09 10:10:27.551999744
211,20250509,1132-CM254C,機器學習應用,第 03 節,1122813,楊智絜,24.970736,121.267609,2025-05-09 10:10:21.504000256
212,20250509,1132-CM254C,機器學習應用,第 04 節,1122810,江佳真,24.970858,121.267807,2025-05-09 11:05:13.344000000


In [3]:
import folium

# 設定地圖中心為所有學生簽到位置的平均值
map_center = [df_cleaned["Latitude"].mean(), df_cleaned["Longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=14)

locations = []
# 在地圖上添加標記
for _, row in df_cleaned.iterrows():
    popup_text = f"姓名: {row['Name']}<br>簽到時間: {row['Check-in Time']}"
    location = [row["Latitude"], row["Longitude"]]
    locations.append(location)
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# 自動調整地圖視窗範圍以包含所有標記
if locations:
    m.fit_bounds(locations)

# 保存地圖為 HTML 文件
html_file_path = "test.html"
m.save(html_file_path)

# 提供下載連結
html_file_path


'test.html'